In [1]:
import pandas as pd

In [2]:
df_classifier_test = pd.read_csv('data/submissions/classifier_test.csv').fillna('')

In [3]:
df_classifier_test.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


## pseudo labing from classifier predictions 

In [5]:
df_classifier_test_confidence = pd.read_pickle('data/classifier_test_confidence.pickle')  

In [6]:
df_classifier_test_confidence.head()

,ImageId_ClassId,Confidence
0,004f40c73.jpg,0.016510
1,006f39c41.jpg,0.045337
2,00b7fb703.jpg,0.176907
3,00bbcd9af.jpg,0.001386
4,0108ce457.jpg,0.277736


In [7]:
print(len(df_classifier_test_confidence[df_classifier_test_confidence.Confidence<0.05]))
print(len(df_classifier_test_confidence[df_classifier_test_confidence.Confidence>0.8]))

1404
196


I'll add the segmentation model predictions to the train dataset that have 
- < 5% confidence on negatives and 
- \> 80% confidence on positives

in the classification model.

In [8]:
df_classifier_confident = df_classifier_test_confidence[(df_classifier_test_confidence.Confidence<0.05) | 
                                                        (df_classifier_test_confidence.Confidence>0.8)]

In [9]:
df_classifier_confident = df_classifier_confident.reset_index(drop=True)

In [10]:
#df_classifier_confident.to_csv('data/confident_classification.csv', index=False)

In [11]:
df_train = pd.read_csv('data/train.csv').fillna('')

In [12]:
df_train.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,
2,0002cc93b.jpg_3,
3,0002cc93b.jpg_4,
4,00031f466.jpg_1,


In [18]:
df_submission = pd.read_csv('data/submission.csv').fillna('')

In [19]:
df_submission.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


In [20]:
pseudolabel = []

for ind, row in df_classifier_confident.iterrows():
    if row['Confidence'] < 0.05 or row['Confidence'] > 0.8:
        for i in range(4):
            class_id = row.ImageId_ClassId + '_' + str(i+1)

            rle = df_submission.loc[df_submission.ImageId_ClassId == class_id].EncodedPixels.values
            if len(rle) > 0:
                rle = rle[0]
            else:
                rle = None
            pseudolabel.append({
                'ImageId_ClassId': class_id,
                'EncodedPixels': rle,
            })

In [21]:
df_pseudolabel_test = pd.DataFrame(pseudolabel)

In [22]:
df_pseudolabel_test.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


In [23]:
len(df_pseudolabel_test), len(df_train)

(6400, 50272)

In [24]:
df_pseudolabel_test[df_pseudolabel_test.EncodedPixels != '']

,ImageId_ClassId,EncodedPixels
10,00bbcd9af.jpg_3,72786 8 73034 26 73287 32 73541 35 73796 37 74...
15,0109b68ec.jpg_4,154349 14 154599 21 154851 27 155105 29 155360...
18,010ec96b4.jpg_3,38386 1 38389 3 38393 4 38631 24 38734 5 38871...
30,01b47d973.jpg_3,244685 3 244938 7 245193 9 245448 12 245703 13...
34,01d49cd47.jpg_3,297610 1 297612 10 297705 12 297849 42 297894 ...
...,...,...
6362,fe3116441.jpg_3,344443 78 344539 36 344693 139 344943 146 3451...
6370,ff18d8aa8.jpg_3,31295 5 31548 9 31802 12 32057 14 32311 17 325...
6374,ff4d29e0d.jpg_3,275714 41 275969 71 276225 75 276481 77 276737...
6390,ffaf13d96.jpg_3,76766 33 76990 65 77235 77 77479 89 77725 99 7...


In [25]:
df_pseudo_train = df_train.append(df_pseudolabel_test, ignore_index=True)

In [26]:
df_pseudo_train.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,
2,0002cc93b.jpg_3,
3,0002cc93b.jpg_4,
4,00031f466.jpg_1,


In [27]:
len(df_pseudo_train), len(df_train), len(df_pseudolabel_test)

(56672, 50272, 6400)

In [28]:
df_pseudo_train.to_csv('data/train_pseudolabel.csv', index=False)

### creating pseudolabels from segmentation model (for classification)

In [100]:
df_segmentation_test_confidence = pd.read_pickle('data/segmentation_test_confidence.pickle')  

In [101]:
df_segmentation_test_confidence.head()

,ImageId_ClassId,EncodedPixels,MaxConfidence,MeanConfidence,MedianConfidence,StdConfidence
0,004f40c73.jpg_1,,0.000191,1.411692e-06,1.215099e-07,0.000007
1,004f40c73.jpg_2,,0.000124,3.824979e-07,1.535455e-08,0.000003
2,004f40c73.jpg_3,,0.014290,3.501467e-05,1.083193e-07,0.000423
3,004f40c73.jpg_4,,0.000051,5.676313e-07,9.133067e-09,0.000002
4,006f39c41.jpg_1,,0.045850,1.765962e-05,1.168544e-06,0.000466


In [102]:
len(df_train), len(df_train[df_train.EncodedPixels!='']), len(df_train[df_train.EncodedPixels!=''])/len(df_train)

(50272, 7095, 0.14113224061107574)

In [103]:
i = 0
for index, row in df_segmentation_test_confidence.iterrows():
    if row.MaxConfidence>0.50:
        #print(row.MaxConfidence)
        i+=1
print(i)

1003


In [104]:
1003/len(df_segmentation_test_confidence)

0.1392282065519156

In [115]:
df_segmentation_test = df_segmentation_test_confidence[df_segmentation_test_confidence.MaxConfidence>0.50]

In [116]:
df_segmentation_test = df_segmentation_test_confidence.reset_index(drop=True)

In [117]:
df_segmentation_test.to_csv('data/confident_segmentation.csv', index=False)

In [118]:
pseudolabel = []

for ind, row in df_segmentation_test.iterrows():
    if row['MaxConfidence'] > 0.50:    
        class_id = row.ImageId_ClassId
        rle = df_segmentation_test.loc[df_segmentation_test.ImageId_ClassId == class_id].EncodedPixels.values
        
        if len(rle) > 0:
            rle = rle[0]
        else:
            rle = ''
            #print('hmm')
            #break
        pseudolabel.append({
            'ImageId_ClassId': class_id,
            'EncodedPixels': rle,
        })

In [119]:
df_pseudolabel_test = pd.DataFrame(pseudolabel)

In [120]:
df_pseudolabel_test.head()

,ImageId_ClassId,EncodedPixels
0,00b7fb703.jpg_3,
1,00bbcd9af.jpg_3,72786 8 73034 26 73287 32 73541 35 73796 37 74...
2,00bbcd9af.jpg_4,
3,0109b68ec.jpg_3,
4,0109b68ec.jpg_4,154349 14 154599 21 154851 27 155105 29 155360...


In [121]:
len(df_pseudolabel_test), len(df_train)

(1003, 50272)

In [122]:
df_pseudolabel_test[df_pseudolabel_test.EncodedPixels != '']

,ImageId_ClassId,EncodedPixels
1,00bbcd9af.jpg_3,72786 8 73034 26 73287 32 73541 35 73796 37 74...
4,0109b68ec.jpg_4,154349 14 154599 21 154851 27 155105 29 155360...
5,010ec96b4.jpg_3,38386 1 38389 3 38393 4 38631 24 38734 5 38871...
6,01b47d973.jpg_3,244685 3 244938 7 245193 9 245448 12 245703 13...
7,01d49cd47.jpg_3,297610 1 297612 10 297705 12 297849 42 297894 ...
...,...,...
997,ff4d29e0d.jpg_3,275714 41 275969 71 276225 75 276481 77 276737...
998,ffa10f6f3.jpg_3,282854 9 282884 9 283034 6 283041 1 283071 4 2...
999,ffaf13d96.jpg_3,76766 33 76990 65 77235 77 77479 89 77725 99 7...
1000,ffbf79783.jpg_3,226359 6 226607 23 226827 8 226859 29 227083 8...


In [123]:
df_pseudo_train = df_train.append(df_pseudolabel_test, ignore_index=True)

In [124]:
df_pseudo_train.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,
2,0002cc93b.jpg_3,
3,0002cc93b.jpg_4,
4,00031f466.jpg_1,


In [125]:
len(df_pseudo_train), len(df_train), len(df_pseudolabel_test)

(51275, 50272, 1003)

In [126]:
df_pseudo_train.to_csv('data/train_pseudolabel_segmentation.csv', index=False)